In [1]:
# 导入包
import pandas as pd
import tushare as ts
import time
import datetime
import json
import numpy as np
from tensorflow.keras import callbacks
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras import models
from tensorflow.keras.layers import Dense,Dropout,Conv2D,BatchNormalization,MaxPooling2D,Input,Flatten
import matplotlib.pyplot as plt

In [2]:
# 导入token
with open('./parameters.json','r') as f:
    p=json.load(f)
#     user = p['user']
#     port = p['port']
#     psw = p['password']
#     host = p['host']
    token=p['TU_share_pro_taken']
#     cnnstr = "mysql://" + user + ":" + psw + "@" + host + ":" + str(port) + "/stock?charset=utf8&use_unicode=1"
# engine_ts = create_engine(cnnstr)
ts.set_token(token)

In [3]:
# ----------参数------------------

stocklist=list(pd.read_csv('./DataAnalysis/stock_list.csv').iloc[:,0])
start='20060101'
end='20201130'
backwards = 66
foresee = 5
max_pe=100*1000
max_pb=100*1000
max_mv=100*1000*1000*1000
split_point=20200501
X_train=[]
X_test=[]
Y_train=[]
Y_test=[]

In [4]:
# 读取宏观数据
Marco=pd.read_csv('./DataAnalysis/Macro2016_20201031.csv')
# 处理宏观数据
df3=Marco
## 处理将所有值的部分除以100
df3=df3.set_index('YYYYMMDD')
df3=df3.apply(lambda x: x/100)
df3.reset_index(inplace=True)
df3.head(5)

,YYYYMMDD,nt_yoy,gdp_yoy,m2_yoy,ppi_yoy,1y
0,20060101,0.0189,0.124,0.1921,0.0305,NaN
1,20060102,0.0189,0.124,0.1921,0.0305,NaN
2,20060103,0.0189,0.124,0.1921,0.0305,NaN
3,20060104,0.0189,0.124,0.1921,0.0305,NaN
4,20060105,0.0189,0.124,0.1921,0.0305,NaN


In [5]:
# 单只股票_处理单片数据
def data_calculation(df):   # df 为数据框格式
    dfa=df.copy()
    dfa.reset_index(drop=True,inplace=True)   # 重新索引
    dfa.insert(dfa.shape[1],'factor',np.nan)  #  增加复权列
    dfa.loc[0,'factor']=1   #  设置起始复权值为1
    for i in range(1,len(dfa)):   #  循环计算后续复权值
        dfa.loc[i,'factor']=dfa.loc[i-1,'factor']*(1+dfa.loc[i,'price_change'])
    # 重新计算开高低收的值
    dfa['open2']=dfa['open']/dfa['close']*dfa['factor']
    dfa['high2']=dfa['high']/dfa['close']*dfa['factor']
    dfa['low2']=dfa['low']/dfa['close']*dfa['factor']
    ts_code=dfa.loc[0,'ts_code']
    trade_date=dfa.loc[backwards-1,'trade_date']
    dfa.drop(columns=['ts_code','trade_date','open','high','low','close','pre_close','price_change'],inplace=True)  # 删除多余列
    #  计算标签值
    label=dfa.loc[backwards:backwards+foresee-1,'factor'].max()/dfa.loc[backwards-1,'factor']-1
    # dfa.insert(dfa.shape[1],'label_value',label)
    # 将换手率换算成0-1之间
    dfa['turnover_rate']=dfa['turnover_rate']/100
    # 删除尾部5行，确保X值在100
    dfa.drop(dfa.tail(foresee).index,inplace=True)
    #  返回数据框以及标签值
    return dfa, ts_code,trade_date,label

In [6]:
# 涨幅分类
def trend_classfication(label):
    if label>0.05:
        r=3
    elif label>0 and label<=0.05:
        r=2
    elif label>-0.05 and label<=0:
        r=1
    else:
        r=0
    return r

In [7]:
# 单只股票处理
def individual_stock(ts_code):
  # 获取股票价格历史
    daily = ts.pro_bar(ts_code=ts_code, start_date=start, end_date=end,
                          asset='E', adj='qfq', freq='D')
    daily.sort_values(by='trade_date',inplace=True)
  # 获取股票日KPI
    pro = ts.pro_api()
    daily_basic=pro.daily_basic(ts_code=ts_code, start_date=start, end_date=end,
                          fields='ts_code,trade_date,turnover_rate,pe_ttm,pb,total_mv')
    daily_basic.sort_values(by='trade_date',inplace=True)
    # 处理价格历史
    df1=daily
    df1.reset_index(drop=True,inplace=True)
    df1['price_change']=(df1['close']-df1['pre_close'])/df1['pre_close']
    df1.drop(columns=['vol','amount','change','pct_chg'],inplace=True)
    # 处理日KPI
    df2=daily_basic
    ## 处理指标，都除以最大数字
    df2['pe_ttm']=df2.pe_ttm.apply(lambda x: x/max_pe)
    df2['pb']=df2.pb.apply(lambda x: x/max_pb)
    df2['total_mv']=df2.total_mv.apply(lambda x: x/max_mv)
    # 合并数据
    df=pd.merge(df1,df2,how='left',on=['ts_code','trade_date'],left_index=False,right_index=False,copy=True)
    df['trade_date']=df['trade_date'].astype('int64')
    # 合并宏观数据
    df=pd.merge(df,df3,how='left',left_on='trade_date',right_on='YYYYMMDD',left_index=False,right_index=False,copy=True)
    # 处理合并后的数据
    df.fillna(method='bfill',axis=0,inplace=True) # 纵向填充，用后面的值填充前面，用以补充最早缺失的数据/数据排序为从早到晚
    df.fillna(method='ffill',axis=0,inplace=True) # 纵向填充，用前面的值填充后面，用以补充最近缺失的数据/数据排序为从早到晚
    df.drop(columns=['YYYYMMDD'],inplace=True)
  # 循环存入单片数据
    if len(df)<(backwards+foresee)*2:
        print(f'stock:{ts_code}数据量不够！')
    else:
        for i in range(len(df)-backwards-foresee):
            ddd,ts_code,trade_date,label = data_calculation(df[i:i+backwards+foresee])
            if trade_date<split_point:
                X_train.append(np.array(ddd))
                Y_train.append(trend_classfication(label))
            else:
                X_test.append(np.array(ddd))
                Y_test.append(trend_classfication(label))        
                # print(f'stock:{ts_code}处理完毕!')

In [8]:
i=1
for stock in stocklist:
    try:
        individual_stock(stock)
    except:
        print("股票处理失败：",stock)
        time.sleep(5)
    print(f'第{i}只股票{stock}处理完毕')
    i=i+1
    # if i>3:
    #   break

第1只股票300720.SZ处理完毕
第2只股票600678.SH处理完毕
第3只股票000955.SZ处理完毕
第4只股票002883.SZ处理完毕
第5只股票600621.SH处理完毕
第6只股票300357.SZ处理完毕
第7只股票300701.SZ处理完毕
第8只股票300617.SZ处理完毕
第9只股票300696.SZ处理完毕
第10只股票002928.SZ处理完毕
第11只股票002677.SZ处理完毕
第12只股票300286.SZ处理完毕
第13只股票300461.SZ处理完毕
第14只股票300606.SZ处理完毕
第15只股票300661.SZ处理完毕
第16只股票300595.SZ处理完毕
第17只股票603360.SH处理完毕
第18只股票002318.SZ处理完毕
第19只股票603040.SH处理完毕
第20只股票300558.SZ处理完毕
第21只股票300726.SZ处理完毕
第22只股票300299.SZ处理完毕
第23只股票600695.SH处理完毕
第24只股票603505.SH处理完毕
第25只股票300624.SZ处理完毕
第26只股票300699.SZ处理完毕
第27只股票600605.SH处理完毕
第28只股票300630.SZ处理完毕
第29只股票002810.SZ处理完毕
第30只股票300522.SZ处理完毕
第31只股票300685.SZ处理完毕
第32只股票600769.SH处理完毕
第33只股票002016.SZ处理完毕
第34只股票002568.SZ处理完毕
第35只股票300729.SZ处理完毕
第36只股票000099.SZ处理完毕
第37只股票300481.SZ处理完毕
第38只股票300585.SZ处理完毕
第39只股票600779.SH处理完毕
第40只股票002812.SZ处理完毕
第41只股票300690.SZ处理完毕
第42只股票600371.SH处理完毕
第43只股票300470.SZ处理完毕
第44只股票600167.SH处理完毕
第45只股票300394.SZ处理完毕
第46只股票601100.SH处理完毕
第47只股票603568.SH处理完毕
第48只股票002049.SZ处理完毕
第49只股票002553.SZ处理完毕
第50只股票300390.SZ处理完毕
第51只股票300

In [9]:
X_train=np.array(X_train)
X_test=np.array(X_test)
Y_train=np.array(Y_train)
Y_test=np.array(Y_test)

In [10]:
X_train=np.reshape(X_train,X_train.shape+(1,))
X_test=np.reshape(X_test,X_test.shape+(1,))

In [11]:
Y_train = to_categorical(Y_train, 4)
Y_test=to_categorical(Y_test,4)

In [12]:
X_train.shape,Y_train.shape,X_test.shape,Y_test.shape

((492955, 66, 13, 1), (492955, 4), (44527, 66, 13, 1), (44527, 4))

In [13]:
# 搭建模型
model=Sequential()
model.add(Conv2D(64,(9,3),input_shape=(backwards,13,1),activation='relu',data_format='channels_last'))
model.add(MaxPooling2D(pool_size=(8,2)))
model.add(Conv2D(64,(6,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Conv2D(32,(2,2),activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Conv2D(64,(3,3),activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Conv2D(64,(3,3),activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Conv2D(64,(3,3),activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.1))
# model.add(Conv2D(64,(3,3),activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(256))
model.add(Dense(256))
model.add(Dense(128))
model.add(Dense(128))
model.add(Dense(64))
model.add(Dense(64))
model.add(Dense(32))
model.add(Dense(4,activation='softmax'))

In [14]:
# my_callbacks = [
#     callbacks.EarlyStopping(patience=2),
#     callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
#     callbacks.TensorBoard(log_dir='./logs'),
# ]

In [15]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])  # binary_crossentropy # categorical_crossentropy

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 58, 11, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 7, 5, 64)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 2, 3, 64)          73792     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 1, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 256)               16640     
_________________________________________________________________
dense_1 (Dense)              (None, 256)               6

In [17]:
# 训练数据
history=model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=200,batch_size=64,verbose=1) # ,callbacks=my_callbacks

Epoch 1/200
7703/7703 [==============================] - 677s 88ms/step - loss: 1.0993 - accuracy: 0.4958 - val_loss: 1.1202 - val_accuracy: 0.4529
Epoch 2/200
7703/7703 [==============================] - 659s 86ms/step - loss: 1.0876 - accuracy: 0.5020 - val_loss: 1.1166 - val_accuracy: 0.4530
Epoch 3/200
7703/7703 [==============================] - 636s 83ms/step - loss: 1.0813 - accuracy: 0.5058 - val_loss: 1.1242 - val_accuracy: 0.4510
Epoch 4/200
7703/7703 [==============================] - 622s 81ms/step - loss: 1.0764 - accuracy: 0.5080 - val_loss: 1.1269 - val_accuracy: 0.4530
Epoch 5/200
7703/7703 [==============================] - 649s 84ms/step - loss: 1.0726 - accuracy: 0.5085 - val_loss: 1.1341 - val_accuracy: 0.4522
Epoch 6/200
7703/7703 [==============================] - 629s 82ms/step - loss: 1.0696 - accuracy: 0.5099 - val_loss: 1.1527 - val_accuracy: 0.4401
Epoch 7/200
7703/7703 [==============================] - 664s 86ms/step - loss: 1.0672 - accuracy: 0.5105 - val_

7703/7703 [==============================] - 181s 23ms/step - loss: 1.0202 - accuracy: 0.5258 - val_loss: 1.3809 - val_accuracy: 0.3596
Epoch 57/200
7703/7703 [==============================] - 185s 24ms/step - loss: 1.0201 - accuracy: 0.5254 - val_loss: 1.3593 - val_accuracy: 0.3586
Epoch 58/200
7703/7703 [==============================] - 186s 24ms/step - loss: 1.0198 - accuracy: 0.5258 - val_loss: 1.3604 - val_accuracy: 0.3630
Epoch 59/200
7703/7703 [==============================] - 193s 25ms/step - loss: 1.0192 - accuracy: 0.5263 - val_loss: 1.4328 - val_accuracy: 0.3562
Epoch 60/200
7703/7703 [==============================] - 202s 26ms/step - loss: 1.0189 - accuracy: 0.5257 - val_loss: 1.3892 - val_accuracy: 0.3612
Epoch 61/200
7703/7703 [==============================] - 197s 26ms/step - loss: 1.0188 - accuracy: 0.5266 - val_loss: 1.4066 - val_accuracy: 0.3614
Epoch 62/200
7703/7703 [==============================] - 188s 24ms/step - loss: 1.0181 - accuracy: 0.5263 - val_loss: 

Epoch 111/200
7703/7703 [==============================] - 179s 23ms/step - loss: 1.0065 - accuracy: 0.5321 - val_loss: 1.6511 - val_accuracy: 0.3563
Epoch 112/200
7703/7703 [==============================] - 183s 24ms/step - loss: 1.0061 - accuracy: 0.5323 - val_loss: 1.6539 - val_accuracy: 0.3591
Epoch 113/200
7703/7703 [==============================] - 188s 24ms/step - loss: 1.0058 - accuracy: 0.5325 - val_loss: 1.6665 - val_accuracy: 0.3594
Epoch 114/200
7703/7703 [==============================] - 191s 25ms/step - loss: 1.0060 - accuracy: 0.5319 - val_loss: 1.6608 - val_accuracy: 0.3533
Epoch 115/200
7703/7703 [==============================] - 192s 25ms/step - loss: 1.0058 - accuracy: 0.5326 - val_loss: 1.6851 - val_accuracy: 0.3474
Epoch 116/200
7703/7703 [==============================] - 184s 24ms/step - loss: 1.0058 - accuracy: 0.5324 - val_loss: 1.6406 - val_accuracy: 0.3520
Epoch 117/200
7703/7703 [==============================] - 179s 23ms/step - loss: 1.0054 - accuracy:

7703/7703 [==============================] - 481s 62ms/step - loss: 0.9985 - accuracy: 0.5358 - val_loss: 1.7959 - val_accuracy: 0.3399
Epoch 166/200
7703/7703 [==============================] - 477s 62ms/step - loss: 0.9983 - accuracy: 0.5364 - val_loss: 1.7119 - val_accuracy: 0.3560
Epoch 167/200
7703/7703 [==============================] - 486s 63ms/step - loss: 0.9982 - accuracy: 0.5360 - val_loss: 1.7775 - val_accuracy: 0.3438
Epoch 168/200
7703/7703 [==============================] - 475s 62ms/step - loss: 0.9981 - accuracy: 0.5362 - val_loss: 1.9262 - val_accuracy: 0.3415
Epoch 169/200
7703/7703 [==============================] - 464s 60ms/step - loss: 0.9979 - accuracy: 0.5362 - val_loss: 1.8792 - val_accuracy: 0.3327
Epoch 170/200
7703/7703 [==============================] - 464s 60ms/step - loss: 0.9979 - accuracy: 0.5358 - val_loss: 1.7836 - val_accuracy: 0.3497
Epoch 171/200
7703/7703 [==============================] - 499s 65ms/step - loss: 0.9977 - accuracy: 0.5366 - val_

KeyboardInterrupt: 

In [ ]:
# 查看模型收敛状况
plt.plot(history.history['accuracy'],label='train_acc')
plt.plot(history.history['val_accuracy'],label='test_acc')
# plt.plot(history.history['loss'],label='train_loss')
# plt.plot(history.history['val_loss'],label='test_loss')
plt.legend()
plt.show()

In [ ]:
model.save('./DataAnalysis/many_stock_model_1')